In [67]:
import pandas as pd

In [68]:
# read the experimental data
data = pd.read_csv('../data/experimental_data.csv')

# Get the accuracies by subject

In [126]:
# get the correct answers from stimulus list
stimuli = pd.read_csv('../stimuli/Stimuli(Sheet1).csv',sep=';')
has_comprehension_q = stimuli[~(stimuli['Question'].isnull())]
q_answers = has_comprehension_q[['Item','Condition','QuestionCorrect']]
q_answers = q_answers[q_answers['Condition'] == 'A'] # since the answer is the same across conditions, keep only A
q_answers = q_answers.drop(['Condition'],axis=1)
q_answers = q_answers.set_index(['Item']) # reindex by item
q_answers = q_answers.replace({
    'Yes':True,
    'No':False
})

# also get which items actually have comprehension questions
items_w_qs = q_answers.index

      QuestionCorrect
Item                 
6               False
7               False
8                True
13               True
16               True
17               True
19               True
20               True
22               True
23               True
25               True
26              False
27               True
28              False
29              False
30              False
35               True
37              False
39              False
40               True
41              False
43              False
44               True
45               True
46              False
47               True
48              False
50               True
51               True
53               True
54               True
55               True
58               True
59               True
61              False
62               True
63               True
64              False
66              False
67              False
68               True
69              False
71               True
72        

In [118]:
#get the responses of each participant to comprehension questions
comp_resp = data[data['PennElementName'] == 'ComprehensionDecide']
comp_resp = comp_resp[comp_resp['Item'].isin(items_w_qs)]
comp_resp = comp_resp[["MD5 hash of participant's IP address",'Item','Condition','Value']]
comp_resp = comp_resp.set_index(['Item','Condition'])
print(comp_resp.head())
print(comp_resp.iloc[1])

               MD5 hash of participant's IP address Value
Item Condition                                           
78   B             b8b10c5c32c46a8f278dc868ef65fd9d     D
17   C             b8b10c5c32c46a8f278dc868ef65fd9d     K
44   D             b8b10c5c32c46a8f278dc868ef65fd9d     K
75   A             b8b10c5c32c46a8f278dc868ef65fd9d     K
37   C             b8b10c5c32c46a8f278dc868ef65fd9d     D
MD5 hash of participant's IP address    b8b10c5c32c46a8f278dc868ef65fd9d
Value                                                                  K
Name: (17, C), dtype: object


In [112]:
final = pd.merge(comp_resp,q_answers)
final.shape

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

# Get completions by subject

In [52]:
completions = data[(data['PennElementName'] == 'BlankAnswer') & (data['Parameter'] == 'Final')]
completions = completions[["MD5 hash of participant's IP address",'Value','Item','Condition']]
completions

,MD5 hash of participant's IP address,Value,Item,Condition
7,b8b10c5c32c46a8f278dc868ef65fd9d,drive,24,D
18,b8b10c5c32c46a8f278dc868ef65fd9d,cut it,78,B
29,b8b10c5c32c46a8f278dc868ef65fd9d,leave,17,C
38,b8b10c5c32c46a8f278dc868ef65fd9d,pour,34,B
49,b8b10c5c32c46a8f278dc868ef65fd9d,scavenge,44,D
...,...,...,...,...
16353,9f8736428d68ef86c654e1f0f0b1058e,translate them,70,C
16364,9f8736428d68ef86c654e1f0f0b1058e,prepare,59,B
16373,9f8736428d68ef86c654e1f0f0b1058e,stop for a drink,5,D
16382,9f8736428d68ef86c654e1f0f0b1058e,get on the bus,42,C


In [53]:
ips = set(completions["MD5 hash of participant's IP address"])
completions_by_subject = {}
for ip in ips:
    subject_completions = list(completions[completions["MD5 hash of participant's IP address"] == ip]['Value'])
    completions_by_subject[ip] = subject_completions
    
with open('../data/completions_by_subject.csv','w') as f:
    for ip in ips:
        f.write(f'---{ip}---\n')
        f.write('\n'.join(completions_by_subject[ip]))
        f.write('\n\n')

# Get completions by item

In [54]:
CONDITIONS = ('A','B','C','D')
ITEMS = tuple(range(1,81))

In [63]:
with open('../data/completions_by_item.csv','w') as f:
    for item in ITEMS:
        for condition in CONDITIONS:
            item_completions = list(completions[(completions['Condition'] == condition) & (completions['Item'] == item)]['Value'])
            f.write(f'---{item}{condition}---\n')
            f.write('\n'.join(item_completions))
            f.write('\n\n')